### Goal : Generate the configurations for SQLite

##### Libraries

In [12]:
import os, time 
import numpy as np
import pandas as pd
from scipy import stats

##### Runtime configs

See https://phiresky.github.io/blog/2020/sqlite-performance-tuning/ for useful complement to official doc

In [13]:
cols = ["-deserialize", "-memtrace", "-maxsize", "-append", "output"]
dico = dict({"-deserialize" : ["", "-deserialize"],
             "-memtrace" : ["", "-memtrace"],
             "-maxsize" : ["-maxsize 268435456","-maxsize 0"],
             "-append" : ["","-append"],
             "output" : ["-line", "-column", "-ascii", "-html"]})

In [14]:
df = pd.DataFrame({})

nb_conf = 1000
nb_conf_final = 50

for d in dico.keys():
    df[d] = np.random.randint(0,len(dico[d]),nb_conf)

df = df.drop_duplicates()[0:nb_conf_final].reset_index().drop(["index"], axis=1)
df

,-deserialize,-memtrace,-maxsize,-append,output
0,0,1,0,0,3
1,1,0,0,0,0
2,0,1,1,0,2
3,0,0,0,0,2
4,0,0,1,0,1
5,0,0,1,1,0
6,1,0,0,1,1
7,1,1,0,0,0
8,0,0,0,1,0
9,1,1,0,1,1


In [15]:
df.to_csv('runtime_options.csv')

In [30]:
for j in range(nb_conf_final):
    with open("./scripts/"+str(j)+".sh","w") as f:
        f.write('#!/bin/bash\nnumb="'+str(j))
        f.write('"\nlogfilename="./logs/$numb.log"\n')
        f.write("# execute the configuration and measure its time\n")
        vals = df.iloc[j]
        for k in range(16):
            cmd_line = "{ time sqlite3 "
            list_conf = ""
            for i in range(len(vals)):
                cmd_line+=dico[cols[i]][vals[i]]+" "
                val_config = str(dico[cols[i]][vals[i]])
                list_conf+=str(int(val_config == ""))+","*(i!=(len(vals)-1))
            cmd_line+=" TPC-H.db '.read ./queries/"+str(k)+".sql' ; } 2> $logfilename\n"
            f.write(cmd_line)
            f.write("""time"""+str(k)+"""=`grep "real" $logfilename | sed 's/real//;s/\t//'`\n""")
        f.write('''csvLine="$numb,''')
        f.write(list_conf+'''"\n''')
        f.write('''csvLine="$csvLine''')
        for k in range(16):
            f.write(',$time'+str(k))
        f.write('''"\n''')
        f.write('''echo "\n"\n''')
        f.write('''echo $csvLine''')

In [17]:
list_conf

'1,0,0,1,0'